# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Getting the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# checking the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion 

## The <font color=red>event_datafile_new.csv</font> contains the following columns:
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# making connection to a Cassandra instance local machine 
# (127.0.0.1)
try: 
    from cassandra.cluster import Cluster
    cluster = Cluster(['127.0.0.1'])

    # establishing connection
    session = cluster.connect()
except Exception as e:
    print(e)
    


#### Create Keyspace

In [6]:
# Creating a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity 
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )

except Exception as e:
    print(e)
    

#### Set Keyspace

In [7]:
# Setting the keyspace specified above

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Creating queries to ask the following three questions of the data

### 1. Give the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1:  Giving the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4


query = "CREATE TABLE IF NOT EXISTS song_info_by_session"
query = query + "(sessionId int, itemInSession int, artist text, \
                    song text, length float, \
                    PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

> **Explanation**: To see the artist, song title and song's length, I defined sessionId as partition key and  itemInSession as clustring key in Primary key. All provides unique primary key. 

In [9]:
# setting up the CSV file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

## Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO song_info_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        ## Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

> **Explanation**: sessionId and itemInSession data type was changed from strig to integer in order to be able to use where statement in sessionId = 338 AND itemInSession = 4

In [10]:
#!pip install -U prettytable #installing Pretty table for the better visualiztion of table

In [11]:
## SELECT statement

query = "select artist, song, length from song_info_by_session WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    

t = PrettyTable(['Artist', 'Song', 'Length'])
for row in rows:
    t.add_row([row.artist, row.song, row.length])
print(t)

+-----------+---------------------------------+--------------------+
|   Artist  |               Song              |       Length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


In [12]:
## Query 2: Giving only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS user_info_by_session"
query = query + "(userId int, sessionId int, itemInSession int,  \
                    artist text, song text, firstName text, lastName text, \
                    PRIMARY KEY ((userId, sessionId ), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

> **Explanation**: To see the name of artist, song and user (first and last name), I defined (sessionId,userId) as partition key and  itemInSession as clustring key in Primary key. All provides unique primary key. Also itemInSession provides the sorting.

In [13]:
# setting up the CSV file.

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

## Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO user_info_by_session (userId, sessionId, itemInSession,  artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        ## Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

> **Explanation**: userId, sessionId and itemInSession data type was changed from strig to integer in order to be able to use where statement in userid = 10, sessionid = 182

In [14]:
## SELECT statement

query = "select artist, song, firstName, lastName from user_info_by_session WHERE sessionId = 182 AND userId = 10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
t = PrettyTable(['Artist', 'Song', 'FirstName', 'Lastname'])
for row in rows:
    t.add_row([row.artist, row.song, row.firstname, row.lastname])
print(t)

+-------------------+------------------------------------------------------+-----------+----------+
|       Artist      |                         Song                         | FirstName | Lastname |
+-------------------+------------------------------------------------------+-----------+----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie  |   Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie  |   Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie  |   Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie  |   Cruz   |
+-------------------+------------------------------------------------------+-----------+----------+


In [15]:
## Giving every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS user_info_by_song"
query = query + "(song text, userId int, \
                    firstName text, lastName text, \
                    PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

> **Explanation**: To see the user name (first and last)who listened to specified song, I defined song as partition key and  userId as clustring key in Primary key. All provides unique primary key. 

In [16]:
# setting up the CSV file.


file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

## Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO user_info_by_song (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        
        ## Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

> **Explanation**: userId data type was changed from strig to integer to make unique. We can also define as text. The results was the same.

In [17]:
## SELECT statement

query = "select firstName, lastName from user_info_by_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
       
    
t = PrettyTable(['FirstName', 'LastName'])
for row in rows:
    t.add_row([row.firstname, row.lastname])
print(t)

+------------+----------+
| FirstName  | LastName |
+------------+----------+
| Jacqueline |  Lynch   |
|   Tegan    |  Levine  |
|    Sara    | Johnson  |
+------------+----------+


In [18]:
## Dropping the table 

In [19]:
query = "drop table song_info_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


In [20]:
query = "drop table user_info_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


In [21]:
query = "drop table user_info_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


### Closing the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()

### Conclussion

> Sparkify wants to analyze the data they've been collecting on songs and user activity on their new music streaming app. I created a database for understanding what songs users are listening to. I processed the event_datafile_new.csv dataset to created a denormalized dataset. I modeled the data tables keeping in mind the queries I need to run. And then I produced the queries and run them in order to answer the analysis questions. 